In [6]:
import os
import yaml

# Load configuration
config = yaml.safe_load(open("myconfig.yml"))

# Set up Azure OpenAI environment variables with the minimum necessary configuration
os.environ["AZURE_OPENAI_API_KEY"] = config["AZURE_OPENAI_API_KEY"]
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
os.environ["AZURE_OPENAI_ENDPOINT"] = config["AZURE_OPENAI_ENDPOINT"]
os.environ["AZURE_OPENAI_EMBEDDING_DEPLOY"] = config.get("AZURE_OPENAI_EMBEDDING_DEPLOY")
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"  # Using a stable API version that works with most Azure OpenAI deployments

os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = str(config["WANDB_PROJECT"])
os.environ["USER_AGENT"] = "LangChain/1.0 (LLM Agent Memory Tutorial)"


# Memory

![./images/Screenshot 2024-02-21 at 19.22.19.png](<./images/Screenshot 2024-02-21 at 19.22.19.png>)


- **Sensory Memory:** This component of memory captures immediate sensory inputs, like what we see, hear, or feel. In the context of prompt engineering and AI models, a prompt serves as a transient input, similar to a momentary touch or sensation. It's the initial stimulus that triggers the model's processing.

- **Short-Term Memory:** Short-term memory holds information temporarily, typically related to the ongoing task or conversation. In prompt engineering, this equates to retaining the recent chat history. This memory enables the agent to maintain context and coherence throughout the interaction, ensuring that responses align with the current dialogue.

- **Long-Term Memory:** Long-term memory stores both factual knowledge and procedural instructions. In AI models, this is represented by the data used for training and fine-tuning. Additionally, long-term memory supports the operation of RAG frameworks, allowing agents to access and integrate learned information into their responses. It's like the comprehensive knowledge repository that agents draw upon to generate informed and relevant outputs.

### Adding long-term memory

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

In [ ]:
# Get Azure OpenAI configuration from environment variables
deployment_name = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOY"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
api_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    

loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()

# Creating more distinct chunks with less overlap
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Size of each chunk
    chunk_overlap=200  # Reduced overlap between chunks
).split_documents(docs)

# Create embeddings object separately for clarity
embeddings = AzureOpenAIEmbeddings(
    deployment=deployment_name,
    openai_api_key=api_key,
    azure_endpoint=api_endpoint,
    validate_base_url=True
)

# Create vector store
vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

[Document(metadata={'source': 'https://neurons-lab.com/', 'title': 'Neurons Lab | Home', 'description': "We're an AI-exclusive consultancy delivering AI transformation services to guide enterprises into the new era of AI. +100 clients supported.", 'language': 'en-US'}, page_content="\n\n\n\n\n\nNeurons Lab | Home\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Services        \n\n            Services        \n\n\n\n\n\n\n\n\n\n\n                    AI Transformation Program                \n\n\n                    A complete service suite to guide you into a new era of AI                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        All services    \n\n        All services    \n\n\n\n\n\n\n\n        Executive AI Alignment    \n\n\n            Align your leadership team on the AI transformation journey and create a clear, actionable roadmap for success.         \n \n\n\n        AI Strategy & G

In [11]:
results = retriever.invoke("what are the projects in healthcare?")
results

[Document(id='ff4cbda1-b777-4580-bce1-b3cf0c03bf39', metadata={'source': 'https://neurons-lab.com/', 'title': 'Neurons Lab | Home', 'description': "We're an AI-exclusive consultancy delivering AI transformation services to guide enterprises into the new era of AI. +100 clients supported.", 'language': 'en-US'}, page_content='Insurance            \n\n\n\n\n                    Global Insurance Company                \n\n                    Leading Insurer Automates Medical Insurance Claim Analysis to Detect Fraud with AI                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        See our case studies    \n\n        See our case studies    \n\n\n\n\n\n\n\n\n\n\n\nDrive Success with Services at Each Stage of the AI Journey\n\n\n\n\n \n\n\n\n\n\n\n\n\n\nThe AI Transformation Program\nWe deliver AI transformation services to guide enterprises into the new era of AI. Our approach covers the complete AI spectrum, combining leadership alignment with technology integration to deliver measurable o

### Adding short-term memory

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [15]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [20]:
from langchain_community.chat_models import AzureChatOpenAI

# Get Azure OpenAI configuration from environment variables
deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

azure_llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    api_version=api_version,
    api_key=api_key,
    temperature=0
)

tools = [retriever_tool]
agent = create_openai_functions_agent(
    azure_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [21]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hi Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hi Bob! How can I assist you today?'}

In [22]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob! 😊

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Bob! 😊'}